In [20]:
import pandas as pd
import numpy as np
import time as time
import statistics
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [21]:
data = pd.read_csv(r'G:\My Drive\FH_Technikum\MSC\Semester_2_SS2022\DAS\ComparativeExperimentation\heart_failure_clinical_records_dataset.csv')
data.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


Split data into training and testdata as well as columns that should be predicted (y) and columns that contain data that will be used to predict (X). (= holdout method)

The columns that should be predicted (target/dependent) must be excluded from the trainingsdata to not influence the created modle.

The dependent (to be predicted) data is located in column 54 (Forest Cover Type Classes => values from 1 to 7)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,:'smoking'], data.loc[:,'DEATH_EVENT':], test_size=0.33, random_state=547998)
print("X_train: " + str(X_train.shape))
print("X_test: " + str(X_test.shape))
print("y_train: " + str(y_train.shape))
print("y_test: " + str(y_test.shape))

X_train: (200, 11)
X_test: (99, 11)
y_train: (200, 1)
y_test: (99, 1)


According to this https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data/discussion/178372 discussion regarding the data (the time column), it should be excluded because it is directly connected to the prediction variable DEATH_EVENT.

## Decision Tree
I chose to vary the parameters for min_samples_splits and min_samples_leafs to see the differences between the different values since they seem to be the most promising to have an impact on the results.

In [23]:
# result analysis helper lists
training_times = []
test_times = []
accuracy_measures = []
weithged_f1_measures = []

# algo input parameter variation lists
min_samples_splits = [2, 50, 100, 1000]
min_samples_leafs = [1, 50, 100, 1000]

for min_samples_split in min_samples_splits:
    for min_samples_leaf in min_samples_leafs:
        algo = DecisionTreeClassifier(criterion='gini', splitter='best', min_samples_split=min_samples_split, random_state=547998)

        start_training = time.time()
        modle = algo.fit(X=X_train, y=y_train)
        training_times.append(time.time() - start_training)

        start_testing = time.time()
        y_pred = modle.predict(X=X_test)
        test_times.append(time.time() - start_testing)

        accuracy_measures.append(accuracy_score(y_true=y_test, y_pred=y_pred))
        weithged_f1_measures.append(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

        print("Run-parameters min_samples_split: [" + str(min_samples_split) + "] min_samples_leaf: [" + str(min_samples_leaf) + "]")
        print("-------------------------------------------")
        print("training time: " + str(training_times[-1]) + " seconds")
        print("testing time: " + str(test_times[-1]) + " seconds")

        print("accuracy: " + str(accuracy_measures[-1]))
        print("micro f-score: " + str(f1_score(y_true=y_test, y_pred=y_pred, average='micro')))
        print("macro f-score: " + str(f1_score(y_true=y_test, y_pred=y_pred, average='macro')))
        print("weighted f-score: " + str(weithged_f1_measures[-1]))
        print("-------------------------------------------")

        # crosschecking results
        # print(classification_report(y_true=y_test, y_pred=y_pred))

mean_training_time = statistics.mean(training_times)
mean_testing_time = statistics.mean(test_times)
mean_accuracy_measure = statistics.mean(accuracy_measures)
mean_weighted_f1_measure = statistics.mean(weithged_f1_measures)

print("mean training time: " + str(mean_training_time))
print("mean testing time: " + str(mean_testing_time))
print("mean accuracy_measures: " + str(mean_accuracy_measure))
print("mean weighted_f1_measures: " + str(mean_weighted_f1_measure))

dt_mean_training_time = mean_training_time
dt_mean_testing_time = mean_testing_time
dt_mean_accuracy_measure = mean_accuracy_measure
dt_mean_weighted_f1_measure = mean_weighted_f1_measure


Run-parameters min_samples_split: [2] min_samples_leaf: [1]
-------------------------------------------
training time: 0.0059909820556640625 seconds
testing time: 0.0010046958923339844 seconds
accuracy: 0.696969696969697
micro f-score: 0.696969696969697
macro f-score: 0.641304347826087
weighted f-score: 0.699824330259113
-------------------------------------------
Run-parameters min_samples_split: [2] min_samples_leaf: [50]
-------------------------------------------
training time: 0.003996372222900391 seconds
testing time: 0.001003265380859375 seconds
accuracy: 0.696969696969697
micro f-score: 0.696969696969697
macro f-score: 0.641304347826087
weighted f-score: 0.699824330259113
-------------------------------------------
Run-parameters min_samples_split: [2] min_samples_leaf: [100]
-------------------------------------------
training time: 0.0020127296447753906 seconds
testing time: 0.0029931068420410156 seconds
accuracy: 0.696969696969697
micro f-score: 0.696969696969697
macro f-sco

## Perceptron
I chose to vary the value for perameter alpha according to this article (https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html) and the value for parameter penalty, since the penalty for a failed attempt seems to have significant impact on the results.

In [24]:
# result analysis helper lists
training_times = []
test_times = []
accuracy_measures = []
weithged_f1_measures = []

# algo input parameter variation lists
alphas = np.logspace(-1, 1, 5)
penalties = ['l2', 'l1']

for alpha in alphas:
    for penalty in penalties:
        algo = Perceptron(alpha=alpha, penalty=penalty, random_state=547998)

        start_training = time.time()
        modle = algo.fit(X=X_train, y=y_train.values.ravel())
        training_times.append(time.time() - start_training)

        start_testing = time.time()
        y_pred = modle.predict(X=X_test)
        test_times.append(time.time() - start_testing)

        accuracy_measures.append(accuracy_score(y_true=y_test, y_pred=y_pred))
        weithged_f1_measures.append(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

        print("Run-parameters penaltiy: [" + str(penalty) + "] alpha: [" + str(alpha) + "]")
        print("-------------------------------------------")
        print("training time: " + str(training_times[-1]) + " seconds")
        print("testing time: " + str(test_times[-1]) + " seconds")

        print("accuracy: " + str(accuracy_measures[-1]))
        print("micro f-score: " + str(f1_score(y_true=y_test, y_pred=y_pred, average='micro')))
        print("macro f-score: " + str(f1_score(y_true=y_test, y_pred=y_pred, average='macro')))
        print("weighted f-score: " + str(weithged_f1_measures[-1]))
        print("-------------------------------------------")

        # crosschecking results
        # print(classification_report(y_true=y_test, y_pred=y_pred))

mean_training_time = statistics.mean(training_times)
mean_testing_time = statistics.mean(test_times)
mean_accuracy_measure = statistics.mean(accuracy_measures)
mean_weighted_f1_measure = statistics.mean(weithged_f1_measures)

print("mean training time: " + str(mean_training_time))
print("mean testing time: " + str(mean_testing_time))
print("mean accuracy_measures: " + str(mean_accuracy_measure))
print("mean weighted_f1_measures: " + str(mean_weighted_f1_measure))

p_mean_training_time = mean_training_time
p_mean_testing_time = mean_testing_time
p_mean_accuracy_measure = mean_accuracy_measure
p_mean_weighted_f1_measure = mean_weighted_f1_measure

Run-parameters penaltiy: [l2] alpha: [0.1]
-------------------------------------------
training time: 0.002000093460083008 seconds
testing time: 0.0 seconds
accuracy: 0.29292929292929293
micro f-score: 0.29292929292929293
macro f-score: 0.22656250000000003
weighted f-score: 0.13273358585858588
-------------------------------------------
Run-parameters penaltiy: [l1] alpha: [0.1]
-------------------------------------------
training time: 0.0044209957122802734 seconds
testing time: 0.0013239383697509766 seconds
accuracy: 0.7070707070707071
micro f-score: 0.7070707070707071
macro f-score: 0.4142011834319526
weighted f-score: 0.5857390472775087
-------------------------------------------
Run-parameters penaltiy: [l2] alpha: [0.31622776601683794]
-------------------------------------------
training time: 0.0019927024841308594 seconds
testing time: 0.0009980201721191406 seconds
accuracy: 0.7070707070707071
micro f-score: 0.7070707070707071
macro f-score: 0.4142011834319526
weighted f-score: 

## K-Nearest Neighbors

When running tests with different algorithms, kd-tree algorithm worked the best (fastest). Other algorithms took too long to be reasonably evaluated.

In [25]:
# result analysis helper lists
training_times = []
test_times = []
accuracy_measures = []
weithged_f1_measures = []

# algo input parameter variation lists
neighbors = [3, 5, 10]

for n_neighbors in neighbors:
    algo = KNeighborsClassifier(n_neighbors=n_neighbors, algorithm='kd_tree')

    start_training = time.time()
    modle = algo.fit(X=X_train, y=y_train.values.ravel())
    training_times.append(time.time() - start_training)

    start_testing = time.time()
    y_pred = modle.predict(X=X_test)
    test_times.append(time.time() - start_testing)

    accuracy_measures.append(accuracy_score(y_true=y_test, y_pred=y_pred))
    weithged_f1_measures.append(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    
    print("Run-parameters n_neighbors: [" + str(n_neighbors) + "]")
    print("-------------------------------------------")
    print("training time: " + str(training_times[-1]) + " seconds")
    print("testing time: " + str(test_times[-1]) + " seconds")

    print("accuracy: " + str(accuracy_measures[-1]))
    print("micro f-score: " + str(f1_score(y_true=y_test, y_pred=y_pred, average='micro')))
    print("macro f-score: " + str(f1_score(y_true=y_test, y_pred=y_pred, average='macro')))
    print("weighted f-score: " + str(weithged_f1_measures[-1]))
    print("-------------------------------------------")
    # crosschecking results
    # print(classification_report(y_true=y_test, y_pred=y_pred))

mean_training_time = statistics.mean(training_times)
mean_testing_time = statistics.mean(test_times)
mean_accuracy_measure = statistics.mean(accuracy_measures)
mean_weighted_f1_measure = statistics.mean(weithged_f1_measures)

print("mean training time: " + str(mean_training_time))
print("mean testing time: " + str(mean_testing_time))
print("mean accuracy_measures: " + str(mean_accuracy_measure))
print("mean weighted_f1_measures: " + str(mean_weighted_f1_measure))

knn_mean_training_time = mean_training_time
knn_mean_testing_time = mean_testing_time
knn_mean_accuracy_measure = mean_accuracy_measure
knn_mean_weighted_f1_measure = mean_weighted_f1_measure

Run-parameters n_neighbors: [3]
-------------------------------------------
training time: 0.002000570297241211 seconds
testing time: 0.008002758026123047 seconds
accuracy: 0.5555555555555556
micro f-score: 0.5555555555555556
macro f-score: 0.4398148148148148
weighted f-score: 0.5452674897119342
-------------------------------------------
Run-parameters n_neighbors: [5]
-------------------------------------------
training time: 0.0009989738464355469 seconds
testing time: 0.002991914749145508 seconds
accuracy: 0.6565656565656566
micro f-score: 0.6565656565656566
macro f-score: 0.5185926773455377
weighted f-score: 0.6253264914592145
-------------------------------------------
Run-parameters n_neighbors: [10]
-------------------------------------------
training time: 0.003000020980834961 seconds
testing time: 0.004000663757324219 seconds
accuracy: 0.696969696969697
micro f-score: 0.696969696969697
macro f-score: 0.4907407407407407
weighted f-score: 0.6249532360643472
---------------------

In [26]:
headers = ["Coverage", "Accuracy", "F1", "Training time", "Testing time"]

table_data = [
    ["K-NN", str(knn_mean_accuracy_measure), str(knn_mean_weighted_f1_measure), str(knn_mean_training_time), str(knn_mean_testing_time)],
    ["Perceptron", str(p_mean_accuracy_measure), str(p_mean_weighted_f1_measure), str(p_mean_training_time), str(p_mean_testing_time)],
    ["Decision Tree", str(dt_mean_accuracy_measure), str(dt_mean_weighted_f1_measure), str(dt_mean_training_time), str(dt_mean_testing_time)],
]

print(tabulate(table_data, headers=headers, tablefmt="grid"))

+---------------+------------+----------+-----------------+----------------+
| Coverage      |   Accuracy |       F1 |   Training time |   Testing time |
+===============+============+==========+=================+================+
| K-NN          |   0.636364 | 0.598516 |      0.00199986 |     0.00499845 |
+---------------+------------+----------+-----------------+----------------+
| Perceptron    |   0.665657 | 0.540439 |      0.00254226 |     0.00103464 |
+---------------+------------+----------+-----------------+----------------+
| Decision Tree |   0.734848 | 0.709041 |      0.00281215 |     0.00112717 |
+---------------+------------+----------+-----------------+----------------+


With the small dataset, execution times are negligible. All three methods require so little time to be trained and tested that the results appear to be produced instantaneously. Decision tree produces models that provide the highest accuracy values. K-NN and Perceptron's accuracy can be considered insufficient. However, even Decision tree's Accuracy and F1 values are not as good as they have been with the large dataset.